# Tongue tracking

This notebook shows the steps for creating a custom DeepLabCut project, labelling the frames to use as training data, training the neural network, analysing the totality of the videos and predicting the trajectory of every point being tracked. 

### Project creation
We define the necessary directory paths for retrieving the videos and create the specific task *jaw_tongue_test-Emilio* with its corresponding [configuration file](./jaw_tongue_test-Emilio-2020-06-30/config.yaml). This file is of crucial importance because it is where we define the main parameters of our neural network, such as its type, the batch size and the specific points to track. In this case we chose a 50Resnet neural network with a batch size of 8 and we decide to track the jaw, the snout, the spout, the tongue and the paw. 


In [1]:
import deeplabcut
import os 
raw_path = '//sv1files.epfl.ch/Petersen-Lab/analysis/Jun_Huang/Film/'

In [2]:
list_animal=next(os.walk(raw_path))[1]
path_animal = list_animal
list_day = [[] for i in range(len(path_animal))]
path_day = list_day
list_session = [[[] for j in range(len(path_day))] for i in range(len(path_animal))]
path_session = list_session

for i in range(len(list_animal)):
    path_animal[i]=raw_path + list_animal[i] + '/'
    list_day[i]=next(os.walk(path_animal[i]))[1]
    for j in range(len(list_day[i])):
        path_day[i][j]=path_animal[i] + list_day[i][j] + '/'
        list_session[i][j] = next(os.walk(path_day[i][j]))[1]
        for k in range(len(list_session[i][j])):
            path_session[i][j][k]=path_day[i][j] + list_session[i][j][k] + '/'
        
path_session_flat = [val for sublist in path_session for val in sublist]
path_list = [val for sublist in path_session_flat for val in sublist]

In [ ]:
path_videos=[]
for path in path_list:
    list_videos = next(os.walk(path))[2]
    path_videos.append(path+list_videos[0]) # get only first video from every session
    #for video in list_videos:
        #path_videos.append(path+video)

In [ ]:
task='jaw_tongue_test' # Enter the name of your experiment Task
experimenter='Emilio' # Enter the name of the experimenter
#path_config_file=deeplabcut.create_new_project(task,experimenter,path_videos, working_directory='C://Users//Jun//Code',copy_videos=False)
path_config_file = 'C://Users//Jun//Code//jaw_tongue_test-Emilio-2020-06-30/config.yaml'

### Frames labelling

For labeling the training data that will be provided to the neural network, we select the firt trial video of every session of every animal, obtining a total of 126 videos for training purposes. From every video we extract 16 frames, giving us a total of 16x126=2016 frames to label. The most important points to select are the tongue and the spout as described in the general notebook. It is important the extracted frames are quite variant for better generalisation of our dataset. In this case we obtain around 30% of the frames with the animal licking, which is a good compromise.  

In [ ]:
%matplotlib inline
deeplabcut.extract_frames(path_config_file) 

In [ ]:
%gui wx
deeplabcut.label_frames(path_config_file)

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

### Network training and evaluation

The previusly prepared training dataset is shuffled and given to the neural network as input for training purposes. This process is extremely computanionally expensive and wont't work if CUDA is not properlly installed on the computer. After the training is finished, the network is tested with some of the frames previously labeled to see if the prediction results are satisfactory. 

In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [ ]:
deeplabcut.train_network(path_config_file)

In [ ]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

### Video analysis
The resulting trained neural network is used with the rest of the videos for automatically tracking the points of interest and plotting its tracectory during every trial. Thsi data will later be use for cross-validating the other behavioutal data collected during the experiments.
The results are stored in hd5 file in the same directory where the video resides. 

In [4]:
analyze_path_videos=[]
for path in path_list:
    list_videos = next(os.walk(path))[2]
    for video in list_videos:
        if len(path+video)<130:
            analyze_path_videos.append(path+video)

In [ ]:
deeplabcut.analyze_videos(path_config_file,analyze_path_videos, save_as_csv=True)

In [ ]:
deeplabcut.create_labeled_video(path_config_file,analyze_path_videos)

In [ ]:
%matplotlib inline
%matplotlib notebook

deeplabcut.plot_trajectories(path_config_file,analyze_path_videos,showfigures=False)
